In [ ]:
import sys
sys.path.append('..')
import pandas as pd
from src.utils.audio_utils import get_audio_features
from tqdm import tqdm

In [ ]:
csv_path = "../data/chartex_final.csv"

sorting data by popularity

In [ ]:
# # Load the CSV file into a DataFrame
# df = pd.read_csv(csv_path)

# # Drop the 'number_of_videos_last_14days' and 'song_name' columns
# df = df.drop(columns=['Unnamed: 0'])

# # Sort the DataFrame by the 'track_name' column
# sorted_df = df.sort_values(by='total_likes_count', ascending=False)

# # Reset the index of the sorted DataFrame
# sorted_df = sorted_df.reset_index(drop=True)

# # Overwrite the original CSV file with the sorted data
# sorted_df.to_csv(csv_path, index=False)

# # Print a message to confirm that the file has output_directory overwritten
# print(f'The file {csv_path} has been overwritten with the sorted data.')

getting Spotify data for most popular song

In [ ]:
# Load the CSV file into a DataFrame
df = pd.read_csv(csv_path)

# Get the first row of data
example_track_data = df.iloc[0]

# Print the resulting row
print(example_track_data)

extracting audio features for example song

In [ ]:
%%time

audio_dir = "../data/track_downloads"

# Get the additional audio features for the track
features = get_audio_features(example_track_data, audio_dir)

extracting for all songs

In [ ]:
def batch_download(df, start_index, end_index):
    for i in range(start_index, end_index):
        print(f'Processing track {i+1} of {len(df)}')
        track_data = df.iloc[i]
        features = get_audio_features(track_data)
        if not features:
            print(f'No features found for track {i+start_index} of {len(df)}, skipping...') 
            continue
        # add features to dataframe
        for feature_names in features.keys():
            df.loc[i, feature_names] = features[feature_names]
        # save dataframe  
        df.to_csv(csv_path, index=False)
    

for i in range(0, len(df), 10):
    batch_download(df, i, i+10)
    

In [ ]:
df.head()